In [ ]:
## This is the code written to fine-tune Sentbert-PP model on PARADE dataset

# Please upload PARADE_dataset-main.zip to colab notebook, zip file present in code base
# Code works on GPU

# **Install Libraries and unzip Parade Dataset**

In [ ]:
!pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4eaabdd8dbf6d193a1ce31193dfb67a3ffe0215f0e7ed1d37367a250f40e9f49
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from transformers import *
import torch
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
from torch.utils.data import DataLoader
from sklearn.utils import shuffle

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


In [ ]:
!unzip PARADE_dataset-main.zip

Archive:  PARADE_dataset-main.zip
   creating: PARADE_dataset-main/
  inflating: PARADE_dataset-main/PARADE_test.txt  
  inflating: PARADE_dataset-main/PARADE_train.txt  
  inflating: PARADE_dataset-main/PARADE_validation.txt  
  inflating: PARADE_dataset-main/README.md  


# **Preprocess PARADE Dataset**

In [ ]:
def pre_process_parade(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()[1:]
        data = {'input1':[], 'input2':[], 'similarity_score':[]}
        for line in lines:
            split_line = line.split('\t')
            data['input1'].append(split_line[3])
            data['input2'].append(split_line[4])
            data['similarity_score'].append(int(split_line[1]))
            
    return data

In [ ]:
train_data_parade = pre_process_parade('./PARADE_dataset-main/PARADE_train.txt')

train_data = {'input1':[], 'input2':[], 'similiarity_score':[]}

train_data['input1'] = train_data_parade['input1']
train_data['input2'] = train_data_parade['input2']
train_data['similarity_score'] = train_data_parade['similarity_score']

# **Finetune Sentbert-PP**

In [ ]:
def finetune_sentbert_pp():

    model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_ex = []
    for i in range(len(train_data['input1'])):
        train_ex.append(InputExample(texts=[train_data['input1'][i], train_data['input2'][i]], label=train_data['similarity_score'][i]))

    train_dataloader = DataLoader(train_ex, shuffle=True, batch_size=32)
    train_loss = losses.ContrastiveLoss(model=model)

    model.fit([(train_dataloader, train_loss)], epochs=3, show_progress_bar=True)
    return model

In [ ]:
model = finetune_sentbert_pp()

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-TinyBERT-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-TinyBERT-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-TinyBERT-L6-v2/pytorch_model.bin
All model checkpoi

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/236 [00:00<?, ?it/s]

Iteration:   0%|          | 0/236 [00:00<?, ?it/s]

Iteration:   0%|          | 0/236 [00:00<?, ?it/s]

# **Save the Fine-tuned model to Google Drive**

In [ ]:
torch.save(model.state_dict(),'finetuned-sentbert-pp-parade-train.pt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/finetuned-sentbert-pp-parade-train.pt" "/content/drive/MyDrive"